In [5]:
import pandas as pd
import requests
import json
import time
from datetime import datetime

from api_key import weather_key2

In [6]:
#Insert dataframe of csv data here
data = pd.read_csv('Resources/aviation_renamed.csv', index_col=False)
weather_key2

'YYPGFXGG79SSPJLE0C9PD1CQ2'

In [7]:
data.head()

,Unnamed: 0,Date,Country,City_State,Make,Model,Injury_Severity,Aircraft_Damage,Latitude,Longitude,Make and Model
0,0,12/31/2019,United States,"Elk, CA",Cessna,175,Non-Fatal,Substantial,39.128611,-123.715833,cessna 175
1,1,12/31/2019,United States,"OLATHE, KS",Mooney,M20S,Fatal(2),Destroyed,38.846111,-94.736111,mooney m20s
2,2,12/31/2019,United States,"Fairbanks, AK",Cessna,170,Non-Fatal,Substantial,64.666945,-148.133334,cessna 170
3,5,12/28/2019,United States,"LAFAYETTE, LA",Piper,PA 31T,Fatal(5),Destroyed,30.176111,-92.007500,piper pa 31t
4,6,12/25/2019,United States,"Headland, AL",Bell,407,Fatal(1),Substantial,31.364167,-85.312500,bell 407


In [21]:
weather_data_df=pd.DataFrame({'Date':[],
                              'Latitude':[],
                              'Longitude':[],
                              'Incident ID':[],
                              'Temperature':[],
                              'Max Temperature':[],
                              'Wind Speed': [],
                              'Visibility':[],
                              'Precipitation':[],
                              'Humidity':[],
                              'Conditions':[],
                              'Cloudcover':[]})
weather_data_df

,Date,Latitude,Longitude,Incident ID,Temperature,Max Temperature,Wind Speed,Visibility,Precipitation,Humidity,Conditions,Cloudcover


In [22]:
#https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations=Herndon,VA,20170&aggregateHours=24&unitGroup=us&shortColumnNames=false&contentType=json&key=YOURAPIKEY

base_url='https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?locations='

for i in range(len(data)):
    date=data['Date'][i].split('/')
    date=date[2]+'-'+date[0]+ '-'+date[1]



    latlong=f'{data.Latitude[i]},{data.Longitude[i]}'
    loc='&aggregateHours=24&unitGroup=us&shortColumnNames=false&contentType=json'
    dt=f'&startDateTime={date}T00%3A00%3A00'
    key=f'&key={weather_key2}'
    url=base_url+latlong+loc+dt+key
    try:
        weather_response = requests.get(url)
        weather_json = weather_response.json()
        weather_json['locations']
        weather_data_single=pd.DataFrame({'Date':[weather_json['locations'][latlong]['values'][0]['datetimeStr']],
                                          'Latitude':[data.Latitude[i]],
                                          'Longitude':[data.Longitude[i]],
                                          'Incident ID':[data['Unnamed: 0'][i]],
                                          'Temperature':[weather_json['locations'][latlong]['values'][0]['temp']],
                                          'Max Temperature':[weather_json['locations'][latlong]['values'][0]['maxt']],
                                          'Wind Speed':[weather_json['locations'][latlong]['values'][0]['wspd']],
                                          'Visibility':[weather_json['locations'][latlong]['values'][0]['visibility']],
                                          'Precipitation':[weather_json['locations'][latlong]['values'][0]['precip']],
                                          'Humidity':[weather_json['locations'][latlong]['values'][0]['humidity']],
                                          'Conditions':[weather_json['locations'][latlong]['values'][0]['conditions']],
                                          'Cloudcover':[weather_json['locations'][latlong]['values'][0]['cloudcover']]
        })
        weather_data_df=weather_data_df.append(weather_data_single)
        time.sleep(.5)
    except:
        print("No value found") #Add and print id of not found
    if i==3:
        break

weather_data_df

,Date,Latitude,Longitude,Incident ID,Temperature,Max Temperature,Wind Speed,Visibility,Precipitation,Humidity,Conditions,Cloudcover
0,2019-12-31T00:00:00-08:00,39.128611,-123.715833,0.0,48.8,58.2,5.2,7.6,0.00,71.77,Clear,14.1
0,2019-12-31T00:00:00-06:00,38.846111,-94.736111,1.0,33.7,42.2,13.9,9.9,0.00,59.91,Clear,0.0
0,2019-12-31T00:00:00-09:00,64.666945,-148.133334,2.0,-3.0,-0.3,4.5,9.3,0.00,76.68,Partially cloudy,74.6
0,2019-12-28T00:00:00-06:00,30.176111,-92.007500,5.0,68.9,76.0,12.8,5.5,0.09,92.13,"Rain, Overcast",89.9


In [ ]:
weather_data_df.to_csv('Resources/weather_raw.csv')